# Energia z OZE dla krajów Unii Europejskiej, Chin, USA, Indii i Rosji

### Wgranie bibliotek i danych

In [2]:
#Biblioteki
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import re

#Zmienienie formatu zapisu danych numerycznych na dwie cyfry po przecinku.
pd.options.display.float_format = '{:.2f}'.format

#Wczytanie danych
df = pd.read_csv('all_energy_statistics.csv')

# Podział kolumny 'commodity_transaction'

źródło kodu: https://www.kaggle.com/code/gabrielapiwar/cleaning-of-commodities-column/edit

In [3]:
#Podział na trzy kolumny
split_commodities = df.commodity_transaction.str.split(" - | – ",  expand=True)

#Przypisanie nazw kolumn
split_commodities.columns = ["commodity", "transaction_type", "additional_transaction_info"]

#Oczyszczenie stringów w kolumnie transaction_type
split_commodities.transaction_type = split_commodities.transaction_type.str.lower().str.strip()

#Poprawienie błędów w pisowni i formatowaniu
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("transformatin", "transformation")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("non energy uses", "consumption for non-energy uses")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace(" /", "/")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("/ ", "/")

#Dodanie trzech nowych kolumn do orginalnych danych
new_df = df.join(split_commodities)

#Usunięcie kolumny 'commodity_transaction'
new_df = new_df.drop(columns = ['commodity_transaction'])

#Zapisanie nowego pliku z danymi
with open("commodity_split_data.csv", "w+") as file:
    file.write(new_df.to_csv())

# Podgląd unikatowych wartości dla każdej kolumny

In [4]:
#Stworzenie słownika z unikatowymi wartościami w kolumnach 
unique_column_values = {}
keys = list(new_df.columns)
for col in keys:
    unique_column_values[col] = new_df[col].unique() 

#Posortowanie kolumny year żeby było czytelniej
unique_column_values['year'].sort()
    
#wypisanie nazw kolumn   
print(unique_column_values.keys(),"\n")

#print("Unikatowe wartości w kolumnie 'country_or_area':\n{}\n".format(unique_column_values['country_or_area']))
print("Unikatowe wartości w kolumnie 'year':\n{}\n".format(unique_column_values['year']))
print("Unikatowe wartości w kolumnie 'unit':\n{}\n".format(unique_column_values['unit']))
print("Unikatowe wartości w kolumnie 'category':\n{}\n".format(unique_column_values['category']))
print("Unikatowe wartości w kolumnie 'commodity':\n{}\n".format(unique_column_values['commodity']))
print("Unikatowe wartości w kolumnie 'transaction_type':\n{}\n".format(unique_column_values['transaction_type']))
print("Unikatowe wartości w kolumnie 'additional_transaction_info':\n{}\n".format(unique_column_values['additional_transaction_info']))

dict_keys(['country_or_area', 'year', 'unit', 'quantity', 'quantity_footnotes', 'category', 'commodity', 'transaction_type', 'additional_transaction_info']) 

Unikatowe wartości w kolumnie 'year':
[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014]

Unikatowe wartości w kolumnie 'unit':
['Metric tons,  thousand' 'Terajoules' 'Kilowatts,  thousand'
 'Kilowatt-hours, million' 'Cubic metres, thousand' 'Metric Tons']

Unikatowe wartości w kolumnie 'category':
['additives_and_oxygenates' 'animal_waste' 'anthracite'
 'aviation_gasoline' 'bagasse' 'biodiesel' 'biogases' 'biogasoline'
 'bitumen' 'black_liquor' 'blast_furnace_gas' 'brown_coal_briquettes'
 'brown_coal' 'charcoal' 'coal_tar' 'coke_oven_coke' 'coking_coal'
 'conventional_crude_oil' 'direct_use_of_geothermal_heat'
 'direct_use_of_solar_thermal_heat'
 'electricity_net_installed_capacity_of_electric_power_plants' 'ethane'
 'falling_water' 'fuel_oil' 'fuelwood

--------------------------------

# Wyodrębnienie elektryczności z OZE

In [5]:
#Pracujemy tylko na jednym commodity (dane się powtarzają między różnymi commodity, aby uprościć wybieramy tylko jedną kategorię)
eco_df = new_df.loc[new_df['commodity'] == 'Electricity']

In [6]:
#Wypisanie transaction type które nas interesuje z Electricity
eco_transaction_type = ['total geothermal production', 'total hydro production', 'total solar production', 
                        'total tide, wave production', 'total wind production', 'total production, main activity', 
                        'gross production', 'gross demand','total production, autoproducer', 'imports', 'exports']

In [7]:
#Stworzenie nowego datasetu tylko z wybranymi transaction type
eco_df = eco_df.loc[eco_df['transaction_type'].isin(eco_transaction_type)]

In [8]:
#Podział na regiony/kraje które nas interesują
EU_country = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia",
              "Finland", "France", "Greece", "Spain", "Netherlands", "Ireland", "Lithuania", "Luxembourg",
              "Latvia", "Malta", "Germany", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Sweden", 
              "Hungary", "Italy", "United Kingdom"]

World = ["United States", "China", "India", "Russian Federation"]

In [9]:
#Stworzenie nowego datasetu tylko z wybranymi regionami/krajami
EU_world_df = eco_df.loc[eco_df['country_or_area'].isin(EU_country + World)]
EU_world_df

,country_or_area,year,unit,quantity,quantity_footnotes,category,commodity,transaction_type,additional_transaction_info
490921,Austria,2014,"Kilowatt-hours, million",0.00,NaN,geothermal,Electricity,total geothermal production,None
490922,Austria,2013,"Kilowatt-hours, million",0.00,NaN,geothermal,Electricity,total geothermal production,None
490923,Austria,2012,"Kilowatt-hours, million",1.00,NaN,geothermal,Electricity,total geothermal production,None
490924,Austria,2011,"Kilowatt-hours, million",1.00,NaN,geothermal,Electricity,total geothermal production,None
490925,Austria,2010,"Kilowatt-hours, million",1.00,NaN,geothermal,Electricity,total geothermal production,None
...,...,...,...,...,...,...,...,...,...
1189456,United States,1994,"Kilowatt-hours, million",3483.00,NaN,wind_electricity,Electricity,total wind production,None
1189457,United States,1993,"Kilowatt-hours, million",3053.00,NaN,wind_electricity,Electricity,total wind production,None
1189458,United States,1992,"Kilowatt-hours, million",2917.00,NaN,wind_electricity,Electricity,total wind production,None
1189459,United States,1991,"Kilowatt-hours, million",3051.00,NaN,wind_electricity,Electricity,total wind production,None


In [10]:
#sprawdzenie jednostek - dane tylko w kilowatach na godzinę, czyli jednostce energii elektrycznej
EU_world_df['unit'].unique()

array(['Kilowatt-hours, million'], dtype=object)

In [11]:
#sprawdzenie category - 
EU_world_df['category'].unique()

array(['geothermal', 'hydro', 'solar_electricity',
       'tide_wave_and_ocean_electricity', 'total_electricity',
       'wind_electricity'], dtype=object)

In [12]:
#suma energii wyprodukowanej w danym kraju, z podziałem na jej rodzaje (suma dla wszystkich lat lub wybrany rok)

#Wybranie rocznika (można pominąć lub wybrać np. kilka lat)
EU_world_df = EU_world_df.loc[EU_world_df['year'] == 2000]

#sumowanie energii osobno dla każdego kraju dla wszystkich roczników
sum_energy_df = EU_world_df[['country_or_area', 'quantity', 'transaction_type']].pivot_table(index='country_or_area',
                                                                                columns='transaction_type',
                                                                               aggfunc = np.sum)
#Podgląd danych
#sum_energy_df

In [13]:
#wartości Nan zamieniamy na 0.0 i usuwamy MultiIndex'owanie
sum_energy_df.fillna(0, inplace=True)
sum_energy_df.columns = sum_energy_df.columns.droplevel()
sum_energy_df.columns.name = None
sum_energy_df.reset_index(inplace=True)

#Podgląd danych
#sum_energy_df

In [14]:
#sumowanie energii z krajów europejskich - stworzenie jednego wiersza EU countries
eu_energy_df = sum_energy_df.loc[sum_energy_df['country_or_area'].isin(EU_country)].sum(numeric_only = True)
eu_energy_df = eu_energy_df.to_frame().transpose()
eu_energy_df['country_or_area'] = 'EU countries'

#Podgląd danych
eu_energy_df

,exports,gross demand,gross production,imports,total geothermal production,total hydro production,"total production, autoproducer","total production, main activity",total solar production,"total tide, wave production",total wind production,country_or_area
0,219330.00,2793601.00,2962276.00,252368.00,4785.00,384568.00,195756.00,2766520.00,118.00,507.00,22224.00,EU countries


In [15]:
#Połączenie EU i World
world_energy_df = sum_energy_df.loc[sum_energy_df['country_or_area'].isin(World)] 

final_energy_df = pd.concat([world_energy_df, eu_energy_df])
final_energy_df.reset_index(inplace=True, drop = True)

#Podgląd danych
final_energy_df

,country_or_area,exports,gross demand,gross production,imports,total geothermal production,total hydro production,"total production, autoproducer","total production, main activity",total solar production,"total tide, wave production",total wind production
0,China,9878.00,1253597.00,1355600.00,1546.00,0.00,222414.00,0.00,1355600.00,0.00,0.00,0.00
1,India,195.00,527212.00,560842.00,1497.00,0.00,74362.00,59638.00,501204.00,0.00,0.00,1582.00
2,Russian Federation,22850.00,794510.00,877766.00,8795.00,58.00,165375.00,47972.00,829794.00,0.00,0.00,2.00
3,United States,14678.00,3818903.00,4052667.00,48592.00,14621.00,279986.00,190551.00,3862116.00,709.00,0.00,5650.00
4,EU countries,219330.00,2793601.00,2962276.00,252368.00,4785.00,384568.00,195756.00,2766520.00,118.00,507.00,22224.00


In [16]:
#dodanie kolumny z sumą całej energi z OZE na kraj
sum_electricity_list = ['total geothermal production', 'total hydro production',
                        'total solar production', 'total tide, wave production']

#Skrót RE oznacza Renewable Energy
final_energy_df['total RE production'] = final_energy_df[sum_electricity_list].sum(axis = 1)

#Podgląd danych
#final_energy_df

In [17]:
#dodanie kolumny z procentem jaki stanowi energia odnawialna w produkcji elektrycznosci
final_energy_df['RE production %'] = (final_energy_df['total RE production'] / final_energy_df['gross production'] * 100)

#dodanie kolumny ile % stanowi produkowana elektryczność w stosunku do zapotrzebowania
final_energy_df['RE production to demand %'] = (final_energy_df['total RE production'] / final_energy_df['gross demand'] * 100)

#dodanie kolumny z różnicą w procentach
final_energy_df['sub'] = final_energy_df['RE production to demand %'] - final_energy_df['RE production %']

#Podgląd danych
#final_energy_df

In [18]:
#Ostateczna wersja tabeli - podgląd danych
final_energy_df

,country_or_area,exports,gross demand,gross production,imports,total geothermal production,total hydro production,"total production, autoproducer","total production, main activity",total solar production,"total tide, wave production",total wind production,total RE production,RE production %,RE production to demand %,sub
0,China,9878.00,1253597.00,1355600.00,1546.00,0.00,222414.00,0.00,1355600.00,0.00,0.00,0.00,222414.00,16.41,17.74,1.34
1,India,195.00,527212.00,560842.00,1497.00,0.00,74362.00,59638.00,501204.00,0.00,0.00,1582.00,74362.00,13.26,14.10,0.85
2,Russian Federation,22850.00,794510.00,877766.00,8795.00,58.00,165375.00,47972.00,829794.00,0.00,0.00,2.00,165433.00,18.85,20.82,1.97
3,United States,14678.00,3818903.00,4052667.00,48592.00,14621.00,279986.00,190551.00,3862116.00,709.00,0.00,5650.00,295316.00,7.29,7.73,0.45
4,EU countries,219330.00,2793601.00,2962276.00,252368.00,4785.00,384568.00,195756.00,2766520.00,118.00,507.00,22224.00,389978.00,13.16,13.96,0.79


In [19]:
#Zapisanie tabeli do pliku csv
with open("sum_eco_energy_data_eu_world.csv", "w+") as file:
    file.write(final_energy_df.to_csv())

-------------------------------------

# Wyodrębnienie energii z biomasy

In [45]:
biomass_commodity_list = ['Bagasse', 'Biodiesel', 'Biogases', 'Biogasoline', 'Peat (for fuel use)', 'Municipal wastes']

In [46]:
biomass_df = new_df.loc[new_df['commodity'].isin(biomass_commodity_list)]
biomass_df = biomass_df.loc[new_df['country_or_area'].isin(EU_country + World)]
#biomass_df = biomass_df.loc[new_df['year'] == 2000]
biomass_df = biomass_df.loc[biomass_df['transaction_type'] == 'total energy supply']
biomass_df

,country_or_area,year,unit,quantity,quantity_footnotes,category,commodity,transaction_type,additional_transaction_info
37544,China,2014,"Metric tons, thousand",40845.00,NaN,bagasse,Bagasse,total energy supply,None
37545,China,2013,"Metric tons, thousand",42812.00,NaN,bagasse,Bagasse,total energy supply,None
37546,China,2012,"Metric tons, thousand",38761.00,NaN,bagasse,Bagasse,total energy supply,None
37547,China,2011,"Metric tons, thousand",34052.00,NaN,bagasse,Bagasse,total energy supply,None
37548,China,2010,"Metric tons, thousand",34442.00,NaN,bagasse,Bagasse,total energy supply,None
...,...,...,...,...,...,...,...,...,...
983831,Sweden,1991,"Metric tons, thousand",1026.00,NaN,peat,Peat (for fuel use),total energy supply,None
983832,Sweden,1990,"Metric tons, thousand",830.00,NaN,peat,Peat (for fuel use),total energy supply,None
983868,United States,1999,"Metric tons, thousand",1.00,NaN,peat,Peat (for fuel use),total energy supply,None
983869,United States,1993,"Metric tons, thousand",24.00,NaN,peat,Peat (for fuel use),total energy supply,None


In [47]:
#Sprawdzenie jednostek
biomass_df['unit'].unique()

#Biogases i Municipal waste są w terajulach, resztę również przekonwertujemy

array(['Metric tons,  thousand', 'Terajoules'], dtype=object)

### Konwersja jednostek
Link do dokumentu z konwersjami: https://mdgs.un.org/unsd/energy/balance/2013/05.pdf
* Bagasse: 1000 T = 7.72 TJ
* Biodiesel: 1000 T = 36.8 TJ
* Biogasoline: 1000 T = 26.8 TJ
* Peat (for fuel use): 1000 T = 9.76 TJ

In [48]:
#Konwersja jednostek z megaton na terajoule 
biomass_df.loc[biomass_df['commodity'] == 'Bagasse', 'quantity'] *= 7.72
biomass_df.loc[biomass_df['commodity'] == 'Biodiesel', 'quantity'] *= 36.8
biomass_df.loc[biomass_df['commodity'] == 'Biogasoline', 'quantity'] *= 26.8
biomass_df.loc[biomass_df['commodity'] == 'Peat (for fuel use)', 'quantity'] *= 9.76
biomass_df['unit'] ='Terajoules' 
biomass_df

,country_or_area,year,unit,quantity,quantity_footnotes,category,commodity,transaction_type,additional_transaction_info
37544,China,2014,Terajoules,315323.40,NaN,bagasse,Bagasse,total energy supply,None
37545,China,2013,Terajoules,330508.64,NaN,bagasse,Bagasse,total energy supply,None
37546,China,2012,Terajoules,299234.92,NaN,bagasse,Bagasse,total energy supply,None
37547,China,2011,Terajoules,262881.44,NaN,bagasse,Bagasse,total energy supply,None
37548,China,2010,Terajoules,265892.24,NaN,bagasse,Bagasse,total energy supply,None
...,...,...,...,...,...,...,...,...,...
983831,Sweden,1991,Terajoules,10013.76,NaN,peat,Peat (for fuel use),total energy supply,None
983832,Sweden,1990,Terajoules,8100.80,NaN,peat,Peat (for fuel use),total energy supply,None
983868,United States,1999,Terajoules,9.76,NaN,peat,Peat (for fuel use),total energy supply,None
983869,United States,1993,Terajoules,234.24,NaN,peat,Peat (for fuel use),total energy supply,None


In [49]:
biomass_df['unit'].unique()

array(['Terajoules'], dtype=object)

In [76]:
#suma energii wyprodukowanej w danym kraju, z podziałem na jej rodzaje (suma dla wszystkich lat lub wybrany rok)

#Wybranie rocznika (można pominąć lub wybrać np. kilka lat)
#EU_world_df = EU_world_df.loc[EU_world_df['year'] == 2000]

#sumowanie energii osobno dla każdego kraju dla wszystkich roczników
sum_biomass_df = biomass_df[['country_or_area', 'quantity', 'commodity']].pivot_table(index='country_or_area',
                                                                                columns='commodity',
                                                                               aggfunc = np.sum)
#Podgląd danych
#sum_biomass_df

In [77]:
#wartości Nan zamieniamy na 0.0 i usuwamy MultiIndex'owanie
sum_biomass_df.fillna(0, inplace=True)
sum_biomass_df.columns = sum_biomass_df.columns.droplevel()
sum_biomass_df.columns.name = None
sum_biomass_df.reset_index(inplace=True)

#Podgląd danych
sum_biomass_df

,country_or_area,Bagasse,Biodiesel,Biogases,Biogasoline,Municipal wastes,Peat (for fuel use)
0,Austria,0.00,172260.80,86592.00,18679.60,203299.00,244.00
1,Belgium,0.00,81107.20,68797.00,11979.60,475812.00,0.00
2,Bulgaria,0.00,13984.00,798.00,938.00,2998.00,0.00
3,China,5513577.68,1602852.34,0.00,623094.72,13750320.00,0.00
4,Croatia,0.00,4673.60,3557.00,268.00,0.00,0.00
5,Cyprus,0.00,4268.80,2442.00,0.00,106.00,0.00
6,Denmark,0.00,32016.00,75400.00,3939.60,776247.00,0.00
7,Estonia,0.00,0.00,2691.00,696.80,7226.00,91158.40
8,Finland,0.00,35880.00,29859.00,22860.40,136063.00,1815242.88
9,France,72.98,818321.60,203149.00,162649.20,1960464.00,0.00


In [78]:
#sumowanie energii z krajów europejskich - stworzenie jednego wiersza EU countries
eu_biomass_df = sum_biomass_df.loc[sum_biomass_df['country_or_area'].isin(EU_country)].sum(numeric_only = True)
eu_biomass_df = eu_biomass_df.to_frame().transpose()
eu_biomass_df['country_or_area'] = 'EU countries'

#Podgląd danych
eu_biomass_df

,Bagasse,Biodiesel,Biogases,Biogasoline,Municipal wastes,Peat (for fuel use),country_or_area
0,52917.61,3749993.60,4669001.00,883998.00,10331086.00,3272518.24,EU countries


In [79]:
#Połączenie EU i World
world_biomass_df = sum_biomass_df.loc[sum_biomass_df['country_or_area'].isin(World)] 

final_biomass_df = pd.concat([world_biomass_df, eu_biomass_df])
final_biomass_df.reset_index(inplace=True, drop = True)

#Podgląd danych
final_biomass_df

,country_or_area,Bagasse,Biodiesel,Biogases,Biogasoline,Municipal wastes,Peat (for fuel use)
0,China,5513577.68,1602852.34,0.00,623094.72,13750320.00,0.00
1,India,12581824.40,0.00,0.00,0.00,0.00,0.00
2,Russian Federation,0.00,0.00,0.00,0.00,0.00,524961.12
3,United States,1985553.12,797640.00,2828828.00,9708434.00,7068572.00,351.36
4,EU countries,52917.61,3749993.60,4669001.00,883998.00,10331086.00,3272518.24


In [80]:
#Dodanie kolumny z sumą całej energii z biomasy
final_biomass_df['total biomass energy'] = final_biomass_df[biomass_commodity_list].sum(axis = 1)

#Podgląd danych
final_biomass_df

,country_or_area,Bagasse,Biodiesel,Biogases,Biogasoline,Municipal wastes,Peat (for fuel use),total biomass energy
0,China,5513577.68,1602852.34,0.00,623094.72,13750320.00,0.00,21489844.74
1,India,12581824.40,0.00,0.00,0.00,0.00,0.00,12581824.40
2,Russian Federation,0.00,0.00,0.00,0.00,0.00,524961.12,524961.12
3,United States,1985553.12,797640.00,2828828.00,9708434.00,7068572.00,351.36,22389378.48
4,EU countries,52917.61,3749993.60,4669001.00,883998.00,10331086.00,3272518.24,22959514.45


In [83]:
#Zmiana Terajouli na Kilowatogodziny i połączenie z tabelą electricity
KWh_df = final_biomass_df.select_dtypes(include=['number']) * 277777.778 / 10**6
final_biomass_df[KWh_df.columns] = KWh_df
final_biomass_df

,country_or_area,Bagasse,Biodiesel,Biogases,Biogasoline,Municipal wastes,Peat (for fuel use),total biomass energy
0,China,3063098.71,890473.52,0.00,346163.73,7639066.67,0.00,11938802.64
1,India,6989902.45,0.00,0.00,0.00,0.00,0.00,6989902.45
2,Russian Federation,0.00,0.00,0.00,0.00,0.00,291645.07,291645.07
3,United States,1103085.07,443133.33,1571571.11,5393574.45,3926984.45,195.20,12438543.61
4,EU countries,29398.67,2083329.78,2593889.45,491110.00,5739492.23,1818065.69,12755285.81
